In [2]:
import re
import os
import pandas as pd

regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
file_paths, file_names, emotions, audios = [], [], [], []
emotion_map = {'anger': 'angry', 'happiness': 'happy', 'sadness': 'sad', 'fear': 'fear',
              'disgust': 'disgust'}

In [3]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1
0.12.1+cu113
cuda


In [4]:
bundle = torchaudio.pipelines.HUBERT_LARGE
extractor = bundle.get_model()
print(extractor.__class__)
print(bundle.sample_rate)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
16000


In [5]:
import os
os.listdir('../emotiondata/emotion_data')

['anger',
 'session_entries.csv',
 'fear',
 'Tools and Documentation',
 'disgust',
 'sadness',
 'happiness']

In [6]:
# folder_list = ['anger', 'disgust', 'fear', 'happiness', 'sadness']
# entries = []
# for folder in folder_list:
#     cur_file_list = os.listdir(f'../emotiondata/emotion_data/{folder}')
#     for i in cur_file_list:
#         if i == 's05 (3).wav':
#             print("found")
#             continue
#         entries.append(i)


In [7]:
# import random
# random.shuffle(entries)
# session = []
# equal_parts = (len(entries)-1)//5 # for equally split the entries into 5 parts
# count = 0
main_data_path = '../emotiondata/emotion_data'

In [8]:
# #### Only Run once
# from tqdm import tqdm 
# folder_map = {'a':'anger', 'd':'disgust', 'f':'fear', 'h':'happiness', 's':'sadness'}

# file_paths = []
# file_names = []
# emotions = []
# # audios = []
# # labels = []


# for i in tqdm(range(len(entries))):
#     entry = entries[i]
#     if "wav" not in entry:
#         continue
#     folder = folder_map[entry[0]]
#     file_path = f'../emotiondata/emotion_data/{folder}/{entry}'
#     emotion = emotion_map[folder]
#     file_paths.append(file_path)
#     file_names.append(entry)
#     emotions.append(emotion)

#     # assign session to it
#     part = (count//equal_parts)%6 + 1
#     if part == 6:
#         part = 5
#     session.append(part)
#     count += 1


# file = pd.DataFrame({'path':file_paths, 'name': file_names, 'emotion': emotions, 'session': session})
# dataframe_path = main_data_path + '/session_entries_hubert.csv'
# file.to_csv(dataframe_path)

In [9]:
# os.listdir('../emotiondata/emotion_data')

# Extract Features using Models

In [10]:
dataframe_path = main_data_path + '/session_entries.csv'
file = pd.read_csv(dataframe_path)[['path', 'name', 'emotion', 'session']]
file.head()

,path,name,emotion,session
0,../emotiondata/emotion_data/happiness/h17 (5).wav,h17 (5).wav,happy,1
1,../emotiondata/emotion_data/happiness/h20 (2).wav,h20 (2).wav,happy,1
2,../emotiondata/emotion_data/disgust/d19 (4).wav,d19 (4).wav,disgust,1
3,../emotiondata/emotion_data/sadness/s12 (4).wav,s12 (4).wav,sad,1
4,../emotiondata/emotion_data/sadness/s01 (4).wav,s01 (4).wav,sad,1


In [11]:
from tqdm import tqdm
audios = []
to_be_discarded = []
discarded_name = []
for i in tqdm(range(len(file['path']))):
    path = file['path'][i]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    
    feature = [f[0] for f in feature]
    audio = torch.stack(feature)
    audios.append(audio)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [15:28<00:00,  1.54s/it]


# Load Data

In [21]:
holdout = 5
train = file[file['session'] != holdout]
train_audios = [audios[i] for i in range(len(audios)) if file['session'][i] != holdout]
test = file[file['session'] == holdout]
test_audios = [audios[i] for i in range(len(audios)) if file['session'][i] == holdout]

In [22]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, audios, labels, label_transform):
        super(MyDataSet).__init__()
        self.audios = audios
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        label = self.label_transform[self.labels[idx]]
        audio = self.audios[idx]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [23]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [24]:
categories = ['angry', 'happy', 'sad', 'fear', 'disgust']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'angry': 0, 'happy': 1, 'sad': 2, 'fear': 3, 'disgust': 4}

In [25]:
from torch.utils.data import DataLoader

train_dataset = MyDataSet(train_audios, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=16, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

test_dataset = MyDataSet(test_audios, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

# 3CNN+LSTM

In [26]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, 
                 bidirectional = False, label_size = 5):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits

# Train Each Layer¶

In [27]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(1024, 24)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|█████▊                                                                                                                                                                                                                                                                                          | 1/50 [00:11<09:12, 11.27s/it]

epoch:1, train accu:0.2896, train loss:1.60


  4%|███████████▌                                                                                                                                                                                                                                                                                    | 2/50 [00:22<09:07, 11.40s/it]

epoch:2, train accu:0.3958, train loss:1.41


  6%|█████████████████▎                                                                                                                                                                                                                                                                              | 3/50 [00:36<09:47, 12.50s/it]

...

epoch:48, train accu:0.9958, train loss:0.02


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 49/50 [08:47<00:10, 10.64s/it]

epoch:49, train accu:0.9896, train loss:0.03


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:59<00:00, 10.78s/it]

epoch:50, train accu:0.9938, train loss:0.02


# Model Test

In [28]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.8064516129032258


In [29]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[-0.1136],
         [-0.1377],
         [ 0.2336],
         [ 0.1521],
         [-0.0580],
         [ 0.1245],
         [ 0.3368],
         [ 0.1190],
         [ 0.0806],
         [ 0.3183],
         [ 0.0618],
         [ 0.1622],
         [-0.1653],
         [-0.0029],
         [ 0.2236],
         [-0.0461],
         [-0.1202],
         [-0.1715],
         [-0.1519],
         [ 0.1115],
         [-0.1562],
         [ 0.0833],
         [-0.2008],
         [-0.0882]]], device='cuda:0')
aggr.bias tensor([0.2713], device='cuda:0')
embed.weight tensor([[ 0.0295,  0.0009, -0.0314,  ...,  0.0254,  0.0174,  0.0109],
        [-0.0311, -0.0350, -0.0229,  ...,  0.0069, -0.0010,  0.0074],
        [ 0.0321, -0.0233,  0.0276,  ...,  0.0576,  0.0289,  0.0159],
        ...,
        [ 0.0374,  0.0004, -0.0099,  ...,  0.0310,  0.0493, -0.0354],
        [-0.0326,  0.0067, -0.0528,  ..., -0.0141,  0.0112,  0.0254],
        [ 0.0496,  0.0211, -0.0059,  ...,  0.0120,  0.0160, -0.0134]]